<a href="https://colab.research.google.com/github/Shereen74/graduation_project/blob/main/Prediction_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content/drive/MyDrive/GP

/content/drive/MyDrive/GP


In [3]:
!pwd

/content/drive/MyDrive/GP


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [5]:
train_daily_x = pd.read_csv('./train_daily_x.csv', keep_default_na=False, index_col=None)
train_daily_y = pd.read_csv('./train_daily_y.csv', keep_default_na=False, index_col=None)
test_daily_x = pd.read_csv('./test_daily_x.csv', keep_default_na=False, index_col=None)
test_daily_y = pd.read_csv('./test_daily_y.csv', keep_default_na=False, index_col=None)

# Preprocessing

In [10]:
# Converting month, day, hour back to timestamp 

train_daily_x['month'] = pd.to_numeric(train_daily_x['month'], errors='coerce')
train_daily_x['day'] = pd.to_numeric(train_daily_x['day'], errors='coerce')
train_daily_x['hour'] = pd.to_numeric(train_daily_x['hour'], errors='coerce')

train_daily_x['timestamp'] = pd.to_datetime('2016-' + train_daily_x['month'].astype(str) + '-' + train_daily_x['day'].astype(str) + ' ' + train_daily_x['hour'].astype(str) + ':00:00')
# Print any rows with invalid dates
print(train_daily_x[train_daily_x['timestamp'].isnull()])

# Remove rows with invalid dates
train_daily_x = train_daily_x.dropna(subset=['timestamp'])

# Reset the index if needed
train_daily_x = train_daily_x.reset_index(drop=True)

Empty DataFrame
Columns: [site_id, day, month, building_id, meter, square_feet, air_temperature, cloud_coverage, dew_temperature, precip_depth_1_hr, sea_level_pressure, wind_speed, IsHoliday, hour, weekend, primary_use_Education, primary_use_Entertainment/public assembly, primary_use_Food sales and service, primary_use_Healthcare, primary_use_Lodging/residential, primary_use_Manufacturing/industrial, primary_use_Office, primary_use_Other, primary_use_Parking, primary_use_Public services, primary_use_Religious worship, primary_use_Retail, primary_use_Services, primary_use_Technology/science, primary_use_Utility, primary_use_Warehouse/storage, timestamp]
Index: []

[0 rows x 32 columns]


In [12]:
# Converting month, day, hour back to timestamp 

test_daily_x['month'] = pd.to_numeric(test_daily_x['month'], errors='coerce')
test_daily_x['day'] = pd.to_numeric(test_daily_x['day'], errors='coerce')
test_daily_x['hour'] = pd.to_numeric(test_daily_x['hour'], errors='coerce')

test_daily_x['timestamp'] = pd.to_datetime('2016-' + test_daily_x['month'].astype(str) + '-' + test_daily_x['day'].astype(str) + ' ' + test_daily_x['hour'].astype(str) + ':00:00')

# Print any rows with invalid dates
print(test_daily_x[test_daily_x['timestamp'].isnull()])

# Remove rows with invalid dates
test_daily_x = test_daily_x.dropna(subset=['timestamp'])

# Reset the index if needed
test_daily_x = test_daily_x.reset_index(drop=True)

Empty DataFrame
Columns: [site_id, day, month, building_id, meter, square_feet, air_temperature, cloud_coverage, dew_temperature, precip_depth_1_hr, sea_level_pressure, wind_speed, IsHoliday, hour, weekend, primary_use_Education, primary_use_Entertainment/public assembly, primary_use_Food sales and service, primary_use_Healthcare, primary_use_Lodging/residential, primary_use_Manufacturing/industrial, primary_use_Office, primary_use_Other, primary_use_Parking, primary_use_Public services, primary_use_Religious worship, primary_use_Retail, primary_use_Services, primary_use_Technology/science, primary_use_Utility, primary_use_Warehouse/storage, timestamp]
Index: []

[0 rows x 32 columns]


In [16]:
train_daily_x = train_daily_x.set_index('timestamp')
test_daily_x = test_daily_x.set_index('timestamp')

In [21]:
train_daily_x['dayofweek'] = train_daily_x.index.dayofweek.astype(np.uint8)
train_daily_x['dayofyear'] = train_daily_x.index.dayofyear
train_daily_x['weekofyear'] = train_daily_x.index.isocalendar().week

test_daily_x['dayofweek'] = test_daily_x.index.dayofweek.astype(np.uint8)
test_daily_x['dayofyear'] = test_daily_x.index.dayofyear
test_daily_x['weekofyear'] = test_daily_x.index.isocalendar().week

In [23]:
train_daily_x

,site_id,day,month,building_id,meter,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,...,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,dayofweek,dayofyear,weekofyear
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,0,1,1,0,0,7432.0,25.0,6.000000,20.0,-0.173913,...,0,0,0,0,0,0,0,4,1,53
2016-01-01 00:00:00,0,1,1,1,0,2720.0,25.0,6.000000,20.0,-0.173913,...,0,0,0,0,0,0,0,4,1,53
2016-01-01 00:00:00,0,1,1,2,0,5376.0,25.0,6.000000,20.0,-0.173913,...,0,0,0,0,0,0,0,4,1,53
2016-01-01 00:00:00,0,1,1,3,0,23685.0,25.0,6.000000,20.0,-0.173913,...,0,0,0,0,0,0,0,4,1,53
2016-01-01 00:00:00,0,1,1,4,0,116607.0,25.0,6.000000,20.0,-0.173913,...,0,0,0,0,0,0,0,4,1,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-31 23:00:00,15,31,8,1444,0,19619.0,22.8,0.363636,17.8,-0.198697,...,0,0,0,0,0,0,0,2,244,35
2016-08-31 23:00:00,15,31,8,1445,0,4298.0,22.8,0.363636,17.8,-0.198697,...,0,0,0,0,0,0,0,2,244,35
2016-08-31 23:00:00,15,31,8,1446,0,11265.0,22.8,0.363636,17.8,-0.198697,...,0,0,0,0,0,0,0,2,244,35


In [25]:
train_daily_x.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13346745 entries, 2016-01-01 00:00:00 to 2016-08-31 23:00:00
Data columns (total 34 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   site_id                                    int64  
 1   day                                        int64  
 2   month                                      int64  
 3   building_id                                int64  
 4   meter                                      int64  
 5   square_feet                                float64
 6   air_temperature                            float64
 7   cloud_coverage                             float64
 8   dew_temperature                            float64
 9   precip_depth_1_hr                          float64
 10  sea_level_pressure                         float64
 11  wind_speed                                 float64
 12  IsHoliday                                  int64  
 13  hour    

# XGBoost Model Creation

In [30]:
features = ['hour', 'day', 'month', 'dayofweek', 'dayofyear', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_speed',
            'square_feet', 'building_id', 'meter']

target = 'meter_reading_log1p'

train_x = train_daily_x[features]
train_y = train_daily_y[target]

test_x = test_daily_x[features]
test_y = test_daily_y[target]

In [32]:
reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree', n_estimators = 3000, early_stopping_rounds=50, objective='reg:squarederror',
                       max_depth=3,
                       learning_rate=0.01)
reg.fit(train_x, train_y, eval_set=[(train_x, train_y), (train_x, train_y)], verbose=100)

[0]	validation_0-rmse:4.14954	validation_1-rmse:4.14954
[100]	validation_0-rmse:2.30947	validation_1-rmse:2.30947
[200]	validation_0-rmse:1.87714	validation_1-rmse:1.87714
[300]	validation_0-rmse:1.76515	validation_1-rmse:1.76515
[400]	validation_0-rmse:1.70889	validation_1-rmse:1.70889
[500]	validation_0-rmse:1.67099	validation_1-rmse:1.67099
[600]	validation_0-rmse:1.64816	validation_1-rmse:1.64816
[700]	validation_0-rmse:1.63054	validation_1-rmse:1.63054
[800]	validation_0-rmse:1.61708	validation_1-rmse:1.61708
[900]	validation_0-rmse:1.60440	validation_1-rmse:1.60440
[1000]	validation_0-rmse:1.59475	validation_1-rmse:1.59475
[1100]	validation_0-rmse:1.58357	validation_1-rmse:1.58357
[1200]	validation_0-rmse:1.57389	validation_1-rmse:1.57389
[1300]	validation_0-rmse:1.56412	validation_1-rmse:1.56412
[1400]	validation_0-rmse:1.55234	validation_1-rmse:1.55234
[1500]	validation_0-rmse:1.54263	validation_1-rmse:1.54263
[1600]	validation_0-rmse:1.53473	validation_1-rmse:1.53473
[1700]	va

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=3000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [33]:
reg2 = xgb.XGBRegressor(base_score=0.5, booster='gbtree', n_estimators = 3000, early_stopping_rounds=20, objective='reg:squarederror',
                       max_depth=5,
                       learning_rate=0.1)
reg2.fit(train_x, train_y, eval_set=[(train_x, train_y), (train_x, train_y)], verbose=100)

[0]	validation_0-rmse:3.84375	validation_1-rmse:3.84375
[100]	validation_0-rmse:1.37270	validation_1-rmse:1.37270
[200]	validation_0-rmse:1.25721	validation_1-rmse:1.25721
[300]	validation_0-rmse:1.17848	validation_1-rmse:1.17848
[400]	validation_0-rmse:1.12182	validation_1-rmse:1.12182
[500]	validation_0-rmse:1.07755	validation_1-rmse:1.07755
[600]	validation_0-rmse:1.04496	validation_1-rmse:1.04496
[700]	validation_0-rmse:1.01502	validation_1-rmse:1.01502
[800]	validation_0-rmse:0.98816	validation_1-rmse:0.98816
[900]	validation_0-rmse:0.96636	validation_1-rmse:0.96636
[1000]	validation_0-rmse:0.94434	validation_1-rmse:0.94434
[1100]	validation_0-rmse:0.92837	validation_1-rmse:0.92837
[1200]	validation_0-rmse:0.91489	validation_1-rmse:0.91489
[1300]	validation_0-rmse:0.90244	validation_1-rmse:0.90244
[1400]	validation_0-rmse:0.89061	validation_1-rmse:0.89061
[1500]	validation_0-rmse:0.88002	validation_1-rmse:0.88002
[1600]	validation_0-rmse:0.87220	validation_1-rmse:0.87220
[1700]	va

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=20,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=3000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [36]:
import os
reg2.save_model('./Models/xgboost2.bin')
# log_dir = './Models'
# os.makedirs(log_dir, exist_ok=True)
# log_file = os.path.join(log_dir), 'xgboost2.log'

In [29]:
features

['hour',
 'day',
 'month',
 'dayofweek',
 'dayofyear',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'sea_level_pressure',
 'wind_speed',
 'square_feet',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'building_id',
 'meter']